In [ ]:
#!pip install einops

In [ ]:
import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import wandb

from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from tqdm.notebook import tqdm

os.environ["WANDB_API_KEY"] = "c609e6f64c7432a3c19cd118b183a082aef411ee"

# Patch embeddings

In [ ]:
class PatchEmbedding(nn.Module):
    """ 
    Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, d_model=768):
        super().__init__()
        
        self.d_model = d_model
        self.in_chans = in_chans
        self.img_size = img_size
        
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_embeddings = nn.Conv2d(3, self.d_model, patch_size, patch_size)

    def forward(self, image):
        b, c, h, w = image.shape
        
        assert h == self.img_size and w == self.img_size, f'Image size must be {self.img_size}x{self.img_size}'
        assert c == self.in_chans, f'Image must have {self.in_chans} channels'
        
        patches = self.patch_embeddings(image).reshape(b, self.d_model, -1).transpose(1, 2)
        
        return patches

In [ ]:
x = torch.randn((2, 3, 224, 224))
PatchEmbedding()(x).shape

# Residual block

In [ ]:
class ResidualBlock(nn.Module):
    
    def __init__(self, func = None) -> None:
        super().__init__()
        
        self.func = func
        if not self.func:
            self.func = lambda x: x
    
    def forward(self, x):
        x = self.func(x) + x
        return x

In [ ]:
x = torch.Tensor([1., 2., 3., 4.])
ResidualBlock(lambda x: x**2)(x)

# Multi Head Attention Block

In [ ]:
class MHABlock(nn.Module):
    def __init__(self, emb_len, num_heads=8, attn_drop=0., out_drop=0.):
        super().__init__()
        
        self.num_heads = num_heads # number of heads
        head_emb = emb_len // num_heads # embeddings length after head
        self.scale = head_emb ** -0.5 # scale param for decrease dispersion

        self.qkv = nn.Linear(emb_len, emb_len * 3, bias=False)
        self.attn_drop = nn.Dropout(attn_drop)
        
        self.out = nn.Sequential(
            nn.Linear(emb_len, emb_len),
            nn.Dropout(out_drop)
        )
        

    def forward(self, x):
        
        QKV = self.qkv(x)
        """
        b - batch
        l - sequence length (number of patches)
        n - 3 (Q K V)
        h - num heads
        hl - seq length after attention
        """
        Q, K, V = rearrange(QKV, 'b l (n h hl) -> n b h l hl', n = 3, h = self.num_heads)

        attention = F.softmax(torch.einsum('bhqo, bhko -> bhqk', Q, K) / self.scale, dim=-1)
        attention = self.attn_drop(attention)
        attention = attention @ V
        attention = rearrange(attention, 'b h l hl -> b l (h hl)')
        
        out = self.out(attention)
        return out


In [ ]:
x = torch.randn((5, 197, 768))
MHABlock(768)(x).shape

# Feed forward block

In [ ]:
class FeedForwardBlock(nn.Module):
    def __init__(self, in_features, mlp_ratio=4, hidden_features=None, out_features=None, drop_rate=0.):
        super().__init__()
        
        if not hidden_features:
            hidden_features = in_features * mlp_ratio
        if not out_features:
            out_features = in_features

        self.linears = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.GELU(),
            nn.Dropout(drop_rate),
            nn.Linear(hidden_features, out_features),
        )

    def forward(self, x):
        x = self.linears(x)
        return x

In [ ]:
x = torch.randn(1, 197, 768)
FeedForwardBlock(768)(x).shape

# Encoder block

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, emb_len, num_heads=8, mlp_ratio=4, drop_rate=0.):
        super().__init__()

        self.first_residual = ResidualBlock(
            nn.Sequential(
                nn.LayerNorm(emb_len),
                MHABlock(emb_len, num_heads, drop_rate, drop_rate),
                nn.Dropout(drop_rate)
            )
        )
        
        self.second_residual = ResidualBlock(
            nn.Sequential(
                nn.LayerNorm(emb_len),
                FeedForwardBlock(emb_len, mlp_ratio),
                nn.Dropout(drop_rate)
            )
        )           

    def forward(self, x):
        
        x = self.first_residual(x)
        x = self.second_residual(x)
        
        return x

In [ ]:
x = torch.randn(1, 197, 768)
block = EncoderBlock(768, 12)
out = block(x)
out.shape

# Transformer class. Stack of EncoderBlocks

In [ ]:
class Transformer(nn.Module):
    def __init__(self, num_layers, emb_len, num_heads=8, mlp_ratio=4, drop_rate=0.):
        super().__init__()
        self.blocks = nn.ModuleList([
            EncoderBlock(emb_len, num_heads, mlp_ratio, drop_rate)
            for i in range(num_layers)])

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x

In [ ]:
x = torch.randn(1, 197, 768)
Transformer(12, 768)(x).shape

# Vision Transformer model

In [ ]:
class ViT(nn.Module):
    """ Vision Transformer with support for patch or hybrid CNN input stage
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=1000,
                 emb_len=768, num_layers=12, num_heads=12, mlp_ratio=4, drop_rate=0.,):
        super().__init__()

        # Присвоение переменных
        ...

        # Path Embeddings, CLS Token, Position Encoding
        self.patch_embeddings = PatchEmbedding(img_size, patch_size, in_chans, emb_len)
        self.cls_token = nn.Parameter(torch.randn((1, 1, emb_len)))
        self.pos_encodings = nn.Parameter(torch.randn((self.patch_embeddings.num_patches + 1, emb_len)))

        # Transformer Encoder
        self.transformer = Transformer(num_layers, emb_len, num_heads, mlp_ratio, drop_rate)

        # Classifier
        self.classifier = nn.Linear(emb_len, num_classes)

    def forward(self, x):
      
        # Path Embeddings, CLS Token, Position Encoding
        b, c, h, w = x.shape
        
        cls_tokens = self.cls_token.expand(b, -1, -1)
        x = self.pos_encodings + torch.cat((cls_tokens, self.patch_embeddings(x)), dim = 1)

        # Transformer Encoder
        x = self.transformer(x)[:, 0, :].squeeze(1)

        # Classifier
        predictions = self.classifier(x)

        return predictions

In [ ]:
x = torch.randn(10, 3, 224, 224)
vit = ViT()
out = vit(x)
out.shape

# Data

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

root_train = '../input/vegetable-image-dataset/Vegetable Images/validation'
root_test = '../input/vegetable-image-dataset/Vegetable Images/test'
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

train_data = datasets.ImageFolder(root_train, transform)
test_data = datasets.ImageFolder(root_test, transform)

train_loader = DataLoader(train_data, 8, True)
test_loader = DataLoader(test_data, 8, False)

# Trainer

In [ ]:
class Trainer:
    
    def __init__(
        self,
        model: nn.Module,
        optimizer: torch.optim.Optimizer,
        scheduler,
        dataloader: torch.utils.data.DataLoader,
        lossfunc: nn.Module,
        epochs: int,
        device: str = 'cuda'
    ) -> None:
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.dataloader = dataloader
        self.lossfunc = lossfunc
        self.epochs = epochs
        self.device = torch.device(device)
    
    
    def train(self) -> nn.Module:
        wandb.watch(self.model, self.lossfunc, log='all', log_freq=100)
        
        self.model.train()
        self.model = self.model.to(self.device)
        
        for epoch in tqdm(range(1, self.epochs + 1), total=self.epochs):
            outputs = []
            for (data, targets) in tqdm(self.dataloader, total=len(self.dataloader)):
                
                loss, acc = self._forward(data, targets)
                self._backward(loss)
                
                outputs.append([loss, acc])
            
            outputs = torch.Tensor(outputs)
            loss, acc = outputs.mean(dim=0).tolist()
            lr = self.scheduler.get_last_lr()[-1]
            wandb.log({"acc": acc, "loss": loss, 'Lr': lr})
            print(f'Эпоха {epoch}: ', acc, loss, lr)
        
        return self.model


    def _forward(self, data: torch.Tensor, targets: torch.Tensor):
        self.optimizer.zero_grad()
        
        data = data.to(self.device)
        targets = targets.to(self.device)
        
        logits = self.model(data)
        
        loss = self.lossfunc(logits, targets)
        acc = torch.sum(logits.argmax(-1) == targets) / len(logits)
        
        return loss, acc
    


    def _backward(self, loss: torch.Tensor) -> None:
        loss.backward()
        self.optimizer.step()
        self.scheduler.step()
        

# Tester

In [ ]:
class Tester:
    
    def __init__(
        self,
        model: nn.Module,
        dataloader: torch.utils.data.DataLoader,
        device: str = 'cuda'
    ) -> None:
        self.model = model
        self.dataloader = dataloader
        self.device = torch.device(device)
    
    
    def test(self) -> nn.Module:
        self.model.eval()
        self.model = self.model.to(self.device)
        
        outputs = []
        for (data, targets) in tqdm(self.dataloader, total=len(self.dataloader)):
            
            acc = self._forward(data, targets)
            outputs.append([acc])
        
        outputs = torch.Tensor(outputs)
        acc = outputs.mean(dim=0).item()
        wandb.log({"Test acc": acc})
        print('Test accuracy: ', acc)
        

    def _forward(self, data: torch.Tensor, targets: torch.Tensor):
        data = data.to(self.device)
        targets = targets.to(self.device)
        
        logits = self.model(data)
        acc = torch.sum(logits.argmax(-1) == targets) / len(logits)
        
        return acc

# Config

In [ ]:
patch_size = 32
num_heads = 16
emb_len = 768
drop_rate = 0.2
max_lr = 0.0002
epochs = 200

# WandB

In [ ]:
print('Initializing WANDB...')
wandb.init(
        project="First ViT",
        name = f"P_{patch_size}-H_{num_heads}-E_{emb_len}-D_{drop_rate}-LR_{max_lr}",
        config={
            'Model': 'Transformer',
            'Optimizer': 'Adam',
            'Scheduler': 'OneCycleLR',
            'architecture': 'RCNN',
            'dataset': 'Custom dataset',
            'epochs': epochs,
        }
    )
print('Initializing succsessful...')

# Model

In [ ]:
print('Initializing model...')
model = ViT(patch_size=patch_size,
            num_heads=num_heads,
            emb_len=emb_len,
            num_classes=15,
            drop_rate=drop_rate)

optim = torch.optim.Adam(model.parameters())
loss = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.OneCycleLR(optim,
                                                total_steps=epochs*len(train_loader),
                                                pct_start=0.2,
                                                max_lr=max_lr)
trainer = Trainer(model, optim, scheduler, train_loader, loss, epochs)
print('Init succsessful')

# Training

In [ ]:
print('Start training')
model = trainer.train()
print('Training is finished')

# Testing

In [ ]:
tester = Tester(model, test_loader)
tester.test()

# Saving model

In [ ]:
torch.save(model.state_dict(), 'Transformer.pth')

In [ ]:
wandb.finish()